# <span style="font-size:1.5em;"> Forecasting Air Quality in New York City
Author: Angela Kim

---

# <span style="font-size:1.2em;"> Contents

<span style="font-size:1.2em;">

- <a href="#Overview">Overview</a>
    
- <a href="#Business Problem">Business Problem</a>
    
- <a href="#Imports">Library Imports & Functions</a>

- <a href="#Data Preparation & Analysis">Data Preparation & Analysis</a>
    
- <a href="#Modeling">Modeling</a>

- <a href="#Visualizations">Visualizations</a>
    
- <a href="#Conclusion">Conclusion</a>

- <a href="#Next Steps">Next Steps</a>

---

# <span style="font-size:1.2em;"> <a id="Overview">Overview</a>

> This project analyzes air pollution data of four major gas pollutants--ground-level ozone (O₃), carbon monoxide (CO), nitrogen dioxide (NO₂), and sulfur dioxide (SO₂)--and creates time series models to forecast future air quality in New York City.
>
> Note: For ease of reference, I will use O3, CO, NO2, and SO2 when naming the pollutants, knowing full well that technically they are not accurate chemical formulas.

# <span style="font-size:1.2em;"> <a id="Business Problem">Business Problem</a>

> Air pollution is a huge problem for everyone. According to the Environmental Defense Fund (EDF), air pollution is currently the biggest environmental risk of premature death. It is highly linked to cardiovascular and respiratory disease and worsens symptoms of susceptible populations.
>
> Not only is air pollution bad for public health, it’s also bad for the economy. Air pollution costs the US roughly 5% of its annual GDP in damages ($790 billion in 2014). The highest costs come from premature deaths. A study by Anthony Heyes, Matthew Neidell, and Soodeh Saberian even suggests that air pollution affects the stock market.
>
> Air pollution also exacerbates the race-class divide. Racial and ethnic minorities are exposed to higher levels of air pollution, especially in highly segregated neighborhoods. Urban areas are more polluted than rural areas, which is where there are denser populations of minorities.
>
> Decreasing air pollution would benefit public health and the economy and contribute to a more equitable society.

# <span style="font-size:1.2em;"> <a id="Imports">Library Imports & Functions</a>

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from math import sqrt
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [3]:
# Functions for EDA

def stationarity_check(df, pollutant):
    """
    Checks stationarity of time series with rolling statistics and Dickey-Fuller test.
    """
    rmean = df.rolling(window=12, center=False).mean()
    rstd = df.rolling(window=12, center=False).std()
    dftest = adfuller(df.iloc[:,0])
    
    # Plot rolling statistics against original
    fig = plt.figure(figsize=(20,4))
    plt.plot(df, color='gray', label='Original')
    plt.plot(rmean, color='blue', label='Rolling Mean')
    plt.plot(rstd, color='magenta', label='Rolling Std')
    plt.title(f'Rolling Mean & Standard Deviation ({pollutant})')
    plt.legend(loc='best')
    plt.show(block=False)
    
    # Print Dickey-Fuller test results
    print('Results of Dickey-Fuller Test: \n')

    dfoutput = pd.Series(dftest[0:4],
                         index=['Test Statistic', 'p-value', '# of Lags Used', '# of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    
    return None


""""""


def decomposition_plot(df):
        """
        Takes time series dataframe and decomposes it in order to observe trend, seasonality, and residuals.
        """
        decomp = seasonal_decompose(df, period=8, model='additive')
        trend = decomp.trend
        seasonal = decomp.seasonal
        residual = decomp.resid
        plt.figure(figsize=(10,7))
        plt.subplot(511)
        plt.plot(df, label='Observed', color='red')
        plt.legend(loc='best')
        plt.subplot(512)
        plt.plot(trend, label='Trend', color='blue')
        plt.legend(loc='best')
        plt.subplot(513)
        plt.plot(seasonal, label='Seasonality', color='orange')
        plt.legend(loc='best')
        plt.subplot(514)
        plt.plot(residual, label='Residuals', color='black')
        plt.legend(loc='best')
        plt.tight_layout()
        
        return decomp

In [4]:
# Functions for time series forecast metrics

def rmse(y_true, y_pred):
    """
    Computes Root Mean Squared Error (RMSE)
    """
    return mse(y_true, y_pred, squared=False) # Can't be bothered to specify "squared=False" each time



def mase(y_true, y_pred, y_train):
    """
    Computes Mean Absolute Scaled Error for univariate time series forecasts.
    *See "Another look at measures of forecast accuracy" | Rob J. Hyndman, Anne B. Koehler (2006)
    """
    e_t = y_true - y_pred
    scale = mean_absolute_error(y_train[1:], y_train[:-1])
    
    return np.mean(np.abs(e_t/scale))

In [5]:
def ARMA_model(df, pollutant):

    y = df[f'{pollutant} AQI']
    aic_scores = []
    train_size = [len(y),500,250,100,50]

    for p in range(6):
        for q in range(6):
            for size in train_size:
                model = ARIMA(endog = y.tail(size), order =(p,0,q))
                fitmodel = model.fit()
                rmse = np.sqrt(mse(fitmodel))
                aic_scores.append(pd.DataFrame(['ARMA', size, (p,0,q), fitmodel.aic, rmse]).T)
                    
    arma_df = pd.concat(aic_scores, axis = 0)
    arma_df.set_axis(['model type', 'train_size', 'order', 'AIC', 'RMSE'], axis = 1, inplace = True)
    return_df = arma_df.sort_values(['train_size','RMSE'], ascending = [False,True]).reset_index(drop = True)
    
    return return_df

# <span style="font-size:1.2em;"> <a id="Data Preparation & Analysis">Data Preparation & Analysis</a>

In [6]:
# Import datasets
O3 = pd.read_csv('data/nycO3.csv')
CO = pd.read_csv('data/nycCO.csv')
NO2 = pd.read_csv('data/nycNO2.csv')
SO2 = pd.read_csv('data/nycSO2.csv')

In [7]:
# Change date columns to datetime and set as index for time series
dflist = [O3, CO, NO2, SO2]

for df in dflist:
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index(['Date'], inplace=True)
    print(df.index.dtype)
    display(df.head())

datetime64[ns]


,O3 AQI
Date,
2000-01-01,25
2000-01-02,38
2000-01-03,31
2000-01-04,29
2000-01-05,24


datetime64[ns]


,CO AQI
Date,
2000-01-01,27.0
2000-01-02,36.0
2000-01-03,38.0
2000-01-04,33.0
2000-01-05,22.0


datetime64[ns]


,NO2 AQI
Date,
2000-01-01,38
2000-01-02,54
2000-01-03,47
2000-01-04,52
2000-01-05,42


datetime64[ns]


,SO2 AQI
Date,
2000-01-01,105.0
2000-01-02,79.0
2000-01-03,99.0
2000-01-04,82.0
2000-01-05,43.0


In [ ]:
# Creating list of daily AQI per pollutant
dailys = []
for df in dflist:
    dailys.append(df.iloc[:])

# Downsampling from days to months
monthlyO3 = dailys[0].resample('M').mean()
monthlyCO = dailys[1].resample('M').mean()
monthlyNO2 = dailys[2].resample('M').mean()
monthlySO2 = dailys[3].resample('M').mean()

# Creating list of monthly AQI per pollutant
monthlys = [monthlyO3, monthlyCO, monthlyNO2, monthlySO2]

# Concatenating individual pollutant dataframes for visualization
df = pd.merge(O3, CO, how='inner', left_index=True, right_index=True)
df = pd.merge(df, NO2, how='inner', left_index=True, right_index=True)
df = pd.merge(df, SO2, how='inner', left_index=True, right_index=True)
df.info()

# Creating list of daily AQI per pollutant
dailys = [df['O3 AQI'], df['CO AQI'], df['NO2 AQI'], df['SO2 AQI']]

# Downsampling from days to months
monthlyO3 = df.copy()['O3 AQI'].resample('M').mean()
monthlyCO = df.copy()['CO AQI'].resample('M').mean()
monthlyNO2 = df.copy()['NO2 AQI'].resample('M').mean()
monthlySO2 = df.copy()['SO2 AQI'].resample('M').mean()

# Creating list of monthly AQI per pollutant
monthlys = [monthlyO3, monthlyCO, monthlyNO2, monthlySO2]

In [ ]:
# Plotting daily AQI
plt.figure(figsize=(20,6), dpi=300)
dailys[0].plot(color='magenta', alpha=0.7)
dailys[1].plot(color='#61d4c1', alpha=0.7)
dailys[2].plot(color='#4527d9', alpha=0.7)
dailys[3].plot(color='#ffd500', alpha=0.7)
plt.title('Daily AQI')
plt.ylabel('AQI')
plt.legend();

In [ ]:
# Plotting monthly AQI
plt.figure(figsize=(20,6), dpi=300)
monthlys[0].plot(color='magenta')
monthlys[1].plot(color='#61d4c1')
monthlys[2].plot(color='#4527d9')
monthlys[3].plot(color='#ffd500')
plt.title('Monthly AQI')
plt.ylabel('AQI')
plt.legend();

In [ ]:
# Log
monthlyO3_log = np.log(monthlyO3)
monthlyCO_log = np.log(monthlyCO)
monthlyNO2_log = np.log(monthlyNO2)
monthlySO2_log = np.log(monthlySO2)

In [ ]:
# Testing with CO
monthlyCO_log.plot(figsize=(20,4))
stationarity_check(monthlyCO_log, 'CO')

In [ ]:
# Moving average
moving_avg = monthlyCO_log.rolling(window=12).mean()
monthlyCO_log.plot(figsize=(22,2))
moving_avg.plot(color='red')
monthlyCO_log_moving_avg_diff = monthlyCO_log - moving_avg
monthlyCO_log_moving_avg_diff.dropna(inplace=True)

stationarity_check(monthlyCO_log_moving_avg_diff, 'CO')

In [ ]:
# Eliminating Trend and Seasonality
# First order differencing 
monthlyCO_log_diff = monthlyCO_log - monthlyCO_log.shift()
plt.plot(monthlyCO_log_diff)
monthlyCO_log_diff.dropna(inplace=True)
stationarity_check(monthlyCO_log_diff, 'CO')

In [ ]:
# Decompose 

# NEED TO FIX #

# monthlyCO_log_decompose = seasonal_decompose(monthlyCO_log).resid
# monthlyCO_log_decompose.dropna(inplace=True)
# stationarity_check(monthlyCO_log_decompose, 'CO')

In [ ]:
# ACF and PACF
lag_acf = acf(monthlyCO_log_diff, nlags=20)
lag_pacf = pacf(monthlyCO_log_diff, nlags=20, method='ols')

In [ ]:
#Plot ACF: 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(monthlyCO_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(monthlyCO_log_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

In [ ]:
#Plot PACF:
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(monthlyCO_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(monthlyCO_log_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [ ]:
i=0
for df in dflist:
    stationarity_check(df, df.columns)
    i+=1

# <span style="font-size:1.2em;"> <a id="Modeling">Modeling</a>

In [ ]:
dd= np.asarray(train.Count)
y_hat = test.copy()
y_hat['naive'] = dd[len(dd)-1]
plt.figure(figsize=(12,8))
plt.plot(train.index, train['Count'], label='Train')
plt.plot(test.index,test['Count'], label='Test')
plt.plot(y_hat.index,y_hat['naive'], label='Naive Forecast')
plt.legend(loc='best')
plt.title("Naive Forecast")
plt.show()


from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(test.Count, y_hat.naive))
print(rms)

RMSE = 43.9164061439

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))

In [ ]:
# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2],12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(monthlyCO_log,
                          order=param,
                          seasonal_order=param_seasonal,
                          enforce_stationarity=False,
                          enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue
        
mod = SARIMAX(monthlyCO_log,
              order=(1,0,1),
              seasonal_order=(1, 0, 1, 12),
              enforce_stationarity=False,
              enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

results.plot_diagnostics(figsize=(15, 12))
plt.show()

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2019-06-30 00:00:00'), dynamic=False)
pred_ci = pred.conf_int()
fig = plt.figure(figsize=(22,6))
ax = monthlyCO_log['2000':].plot(label='Observed')
pred.predicted_mean.plot(ax=ax, label='One-step Ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)

ax.set_xlabel('Date')
ax.set_ylabel('CO AQI Levels')
plt.legend()
plt.show()

In [ ]:
monthlyCO_forecasted = np.exp(pred.predicted_mean)
monthlyCO_truth = np.exp(monthlyCO_log['2015-06-30 00:00:00':])

"""
NEED TO FIX THIS
"""

# Compute the mse and rmse
mse =( (monthlyCO_forecasted - monthlyCO_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print('The RMSE of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

In [ ]:
"""
NEED TO FIX THIS
"""
MASE(monthlyCO_truth, monthlyCO_truth, monthlyCO_forecasted)

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('2015-06-30 00:00:00'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

ax = monthlyCO_log['2000':].plot(label='Observed', figsize=(20, 6))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='k', alpha=.25)

ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('2015-06-30 00:00:00'), monthlyCO_log.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Date')
ax.set_ylabel('CO AQI')

plt.legend()
plt.show()

In [ ]:
# Get forecast 500 steps ahead in future
pred_uc = results.get_forecast(steps=12)

# Get confidence intervals of forecasts
pred_ci = pred_uc.conf_int()

ax = monthlyCO_log.plot(label='Observed', figsize=(20, 6))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('CO AQI')

plt.legend()
plt.show()


np.exp(pred_uc.predicted_mean)

In [ ]:
arma = ARMA_model(O3, 'O3')

# mod_arma = ARMA(O3, order=(1,0), freq='D')
# O3_arma = mod_arma.fit()
# print(O3_arma.summary())

# mod_arma = ARMA(O3, order=(0,1), freq='D')
# O3_arma = mod_arma.fit()
# print(O3_arma.summary())

# mod_arma = ARMA(O3, order=(2,0), freq='D')
# O3_arma = mod_arma.fit()
# print(O3_arma.summary())

# mod_arma = ARMA(O3, order=(0,2), freq='D')
# O3_arma = mod_arma.fit()
# print(O3_arma.summary())

# mod_arma = ARMA(O3, order=(2,1), freq='D')
# O3_arma = mod_arma.fit()
# print(O3_arma.summary())

# mod_arma = ARMA(O3, order=(1,2), freq='D')
# O3_arma = mod_arma.fit()
# print(O3_arma.summary())

# mod_arma = ARMA(O3, order=(2,2), freq='D')
# O3_arma = mod_arma.fit()
# print(O3_arma.summary())

# mod_arima = ARIMA(O3, order=(1,1,1), freq='D')
# O3_arima = mod_arima.fit()
# print(O3_arima.summary())

# mod_sarimax = SARIMAX(O3, order=(1,1,1), freq='D')
# O3_sarimax = mod_sarimax.fit()
# print(O3_sarimax.summary())

# <span style="font-size:1.2em;"> <a id="Visualizations">Visualizations</a>

In [ ]:
df = pd.merge(O3, CO, how='inner', left_index=True, right_index=True)
df = pd.merge(df, NO2, how='inner', left_index=True, right_index=True)
df = pd.merge(df, SO2, how='inner', left_index=True, right_index=True)
df.info()

In [ ]:
# Creating list of daily AQI per pollutant
dailys = [df['O3 AQI'], df['CO AQI'], df['NO2 AQI'], df['SO2 AQI']]

# Downsampling from days to months
monthlyO3 = df.copy()['O3 AQI'].resample('M').mean()
monthlyCO = df.copy()['CO AQI'].resample('M').mean()
monthlyNO2 = df.copy()['NO2 AQI'].resample('M').mean()
monthlySO2 = df.copy()['SO2 AQI'].resample('M').mean()

# Creating list of monthly AQI per pollutant
monthlys = [monthlyO3, monthlyCO, monthlyNO2, monthlySO2]

In [ ]:
# Plotting daily AQI
plt.figure(figsize=(20,6), dpi=300)
dailys[0].plot(color='magenta', alpha=0.7)
dailys[1].plot(color='#61d4c1', alpha=0.7)
dailys[2].plot(color='#4527d9', alpha=0.7)
dailys[3].plot(color='#ffd500', alpha=0.7)
plt.title('Daily AQI')
plt.ylabel('AQI')
plt.legend();

In [ ]:
# Plotting monthly AQI
plt.figure(figsize=(20,6), dpi=300)
monthlys[0].plot(color='magenta')
monthlys[1].plot(color='#61d4c1')
monthlys[2].plot(color='#4527d9')
monthlys[3].plot(color='#ffd500')
plt.title('Monthly AQI')
plt.ylabel('AQI')
plt.legend();

# <span style="font-size:1.2em;"> <a id="Conclusion">Conclusion</a>

> In conclusion, my SARIMA model forecasted air quality in New York City quite well and could even be used in shaping government policy on public health. I would recommend implementing measures to decrease the presence of air pollutants, especially ozone and nitrogen dioxide, as there hasn't been much decrease from 2000.

# <span style="font-size:1.2em;"> <a id="Next Steps">Next Steps</a>

> Given more time and resources, I would like to explore beyond New York City, modeling for other cities and even seeing how cities compare to suburban or rural areas. Another pollutant I'd like to consider is particulate matter. In terms of modeling, it would be interesting to see how well a recurrent neural network would perform.

Function sources:

https://www.kaggle.com/victoraqiao/time-series-forecasting-arima-co-aqi

MASE:
https://github.com/scikit-learn/scikit-learn/issues/18685